In [ ]:
import os
import numpy as np
import tifffile
from scipy.fft import fft2, ifft2, fftshift, ifftshift
from scipy.ndimage import gaussian_filter
from skimage import exposure
from tqdm import tqdm

carpeta_entrada =r"c:\Users\a.ricaurte\Desktop\reconstructed"
carpeta_salida =r"c:\Users\a.ricaurte\Desktop\tajadas_para_volumen"

x0, y0 = 810, 778.5
a, b = 380, 350
factor_expansion = 1.10
a_exp = a * factor_expansion
b_exp = b * factor_expansion


def crear_mascara_eliptica(shape):
    alto, ancho = shape
    Y, X = np.ogrid[:alto, :ancho]
    return (((X - x0) / a_exp)**2 + ((Y - y0) / b_exp)**2) <= 1

def filtro_fft_angular(imagen, umbral_angulo=3, pasos=3):
    f = fft2(imagen)
    fshift = fftshift(f)
    filas, cols = imagen.shape
    cx, cy = cols // 2, filas // 2
    Y, X = np.ogrid[:filas, :cols]
    dx = X - cx
    dy = Y - cy
    angulo = np.arctan2(dy, dx) * 180 / np.pi

    mascara_total = np.ones_like(fshift, dtype=bool)
    for offset in np.linspace(0, umbral_angulo, pasos):
        for a in range(-90, 90, umbral_angulo):
            mascara_local = (np.abs(angulo - (a + offset)) < umbral_angulo / 2)
            mascara_total &= ~mascara_local

    fshift_filtrado = fshift * mascara_total
    imagen_filtrada = np.abs(ifft2(ifftshift(fshift_filtrado)))
    return imagen_filtrada

def procesar_imagen(imagen):
    imagen = imagen.astype(np.float32)
    mascara_elipse = crear_mascara_eliptica(imagen.shape)

    imagen_masc = np.zeros_like(imagen)
    imagen_masc[mascara_elipse] = imagen[mascara_elipse]

    imagen_fft = filtro_fft_angular(imagen_masc)

    gauss_suavizado = gaussian_filter(imagen_fft, sigma=1)
    unsharp = imagen_fft + 0.8 * (imagen_fft - gauss_suavizado)

    normalizada = exposure.rescale_intensity(unsharp, in_range='image', out_range=(0, 1))
    ajustada = exposure.adjust_sigmoid(normalizada, cutoff=0.35, gain=8)
    ajustada = exposure.adjust_gamma(ajustada, gamma=0.95)

    imagen_final = (ajustada * 65535).astype(np.uint16)
    salida = np.zeros_like(imagen_final)
    salida[mascara_elipse] = imagen_final[mascara_elipse]
    return salida


os.makedirs(carpeta_salida, exist_ok=True)
archivos = [f for f in os.listdir(carpeta_entrada) if f.lower().endswith(".tif")]

for archivo in tqdm(archivos, desc="Procesando imágenes"):
    ruta_entrada = os.path.join(carpeta_entrada, archivo)
    ruta_salida = os.path.join(carpeta_salida, archivo)

    imagen = tifffile.imread(ruta_entrada)
    procesada = procesar_imagen(imagen)
    tifffile.imwrite(ruta_salida, procesada, compression=None)






Procesando imágenes: 100%|██████████| 1558/1558 [1:54:01<00:00,  4.39s/it]
